In [12]:
import re

reference_sequence = 'SKSYYDILGVPKSASERQIKKAFHKLAMKYHPDKNKSPDAEAKFREIAEAYETLSDANRRKEYDTLGHSAFTSGKGQRGSGSSFEQSFNFNFDDLFKDFGFFGQNQNTGSKKRFENHFQTRQDGGSSRQRHHFQEFSFGGGLFDDMFEDMEKMFSFSGFDSTNQHTVQTENRFHGSSKHCRTVTQRRGNMVTTYTDCSGQ'
new_sequence = 'KQDYYEILGVSKTAEEREIRKAYKRLAMKYHPDRNQGDKEAEAKFKEIKEAYEVLTDSQKRAAYDQYGHA'
gap_penalty = 2

allowed_characters = {'A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y'}
for character in reference_sequence:
    if character.upper() not in allowed_characters:
        print('incorrect character in reference sequence')
    
for character in new_sequence:
    if character.upper() not in allowed_characters:
        print('incorrect character in new sequence')
    
reference_sequence = '00' + reference_sequence.upper()
new_sequence = '00' + new_sequence.upper()

alignment_matrix = [[[0 for x in range(len(new_sequence))]for y in range(len(reference_sequence))]for z in range(2)]
                    
for x in range(len(new_sequence)):
                    alignment_matrix[0][0][x]=new_sequence[x]
                    alignment_matrix[1][0][x]=new_sequence[x]
for y in range(len(reference_sequence)):
                    alignment_matrix[0][y][0]=reference_sequence[y]
                    alignment_matrix[1][y][0]=reference_sequence[y]
                    
for x in range(len(new_sequence)-2):
                    alignment_matrix[0][1][x+2]=(x+1)*-gap_penalty
for y in range(len(reference_sequence)-2):
                    alignment_matrix[0][y+2][1]=(y+1)*-gap_penalty      
                    
                  
    
file = open('/Users/na395/Desktop/blosum.txt','r')


blosum_matrix = []
line_number=0
for line in file:
    blosum_matrix.append([])
    entrys=line.strip('\n').split('\t')
    entry_number=0
    for entry in entrys:
        blosum_matrix[line_number].append(entry)
    line_number = line_number+1
j = 0
i = 0
h = 0
for x in range(len(new_sequence)-2):
    for y in range(len(reference_sequence)-2):
        j = blosum_matrix[0].index(reference_sequence[y+2])
        i = blosum_matrix[0].index(new_sequence[x+2])
        sxy = int(blosum_matrix[j][i])
        alignment_matrix[1][y+2][x+2]= sxy

                    
        diag  =  int(sxy) + alignment_matrix[0][y+2-1][x+2-1]
        right =  alignment_matrix[0][y+2][x+2-1] - gap_penalty
        down  =  alignment_matrix[0][y+2-1][x+2]- gap_penalty
        if  diag > right and diag > down :
            h = diag
        elif right > down:
            h = right
        else:
            h = down
        alignment_matrix[0][y+2][x+2]= h
        
    
#for each in alignment_matrix:
#    for every in each:
#        print(every)



    file.close()
    
x = len(new_sequence)
y = len(reference_sequence)
previous_choice = 'none'
aligned_reference_sequence = ''
aligned_new_sequence = ''
for cycle in range(x+y):
    choices = ''

    if x == 2 and y ==2:
        break
    elif x == 2:
        aligned_reference_sequence =  reference_sequence[y-1] + aligned_reference_sequence
        aligned_new_sequence =  '-' + aligned_new_sequence
        x = x
        y = y-1
    
    elif y == 2:
        aligned_reference_sequence =  '-' + aligned_reference_sequence
        aligned_new_sequence =  new_sequence[x-1] + aligned_new_sequence
        x = x-1
        y = y
        
    else:
        
        if alignment_matrix[0][y-1][x-1] == alignment_matrix[0][y-1-1][x-1] - gap_penalty:
            choices = choices + 'u'
        if alignment_matrix[0][y-1][x-1] == alignment_matrix[0][y-1-1][x-1-1] + alignment_matrix[1][y-1][x-1]:
            choices = choices + 'd'        
        if alignment_matrix[0][y-1][x-1] == alignment_matrix[0][y-1][x-1-1] - gap_penalty:
            choices = choices + 'l'   

        if choices.find(previous_choice)>-1:
            choice = previous_choice

        else:
            choice = choices[0]

        if choice == 'u':
            aligned_reference_sequence =  reference_sequence[y-1] + aligned_reference_sequence
            aligned_new_sequence =  '-' + aligned_new_sequence
            x = x
            y = y-1
        elif choice == 'd':
            aligned_reference_sequence =  reference_sequence[y-1] + aligned_reference_sequence
            aligned_new_sequence =  new_sequence[x-1] + aligned_new_sequence
            x = x-1
            y = y-1
        elif choice == 'l':
            aligned_reference_sequence =  '-' + aligned_reference_sequence
            aligned_new_sequence =  new_sequence[x-1] + aligned_new_sequence
            x = x-1
            y = y
        else:
            print('error at choice')
        previous_choice = choice

line_length = 40
ref_pos = 0
new_pos = 0
for line in range(int(len(aligned_reference_sequence)/line_length)):    
    ref_chunk = aligned_reference_sequence[0+20*line:line_length+20*line]
    new_chunk = aligned_new_sequence[0+20*line:line_length+20*line]
    
    print('ref\t' + str(ref_pos) + '\t' + ref_chunk + '\t' + str(ref_pos + len(ref_chunk.replace('-','')) -1))
    print('new\t' + str(new_pos) + '\t' + new_chunk + '\t' + str(new_pos + len(new_chunk.replace('-','')) -1) + '\n')
    ref_pos = ref_pos + len(ref_chunk.replace('-',''))
    new_pos = new_pos + len(new_chunk.replace('-',''))
        



ref	0	SKSYYDILGVPKSASERQIKKAFHKLAMKYHPDKNKSPDA	39
new	0	KQDYYEILGVSKTAEEREIRKAYKRLAMKYHPDRNQG-D-	37

ref	40	KAFHKLAMKYHPDKNKSPDAEAKFREIAEAYETLSDANRR	79
new	38	KAYKRLAMKYHPDRNQG-D---K--E-AEA----------	60

ref	80	EAKFREIAEAYETLSDANRRKEYDTLGHSAFTSGKGQRGS	119
new	61	--K--E-AEA----------K---------F---K-----	68

ref	120	KEYDTLGHSAFTSGKGQRGSGSSFEQSFNFNFDDLFKDFG	159
new	69	K---------F---K---------E---------I-KE--	75

ref	160	GSSFEQSFNFNFDDLFKDFGFFGQNQNTGSKKRFENHFQT	199
new	76	----E---------I-KE-----------A---YE--VLT	85

